In [1]:
import numpy as np
import pandas as pd
from selenium import webdriver
from requests_html import HTML
from bs4 import BeautifulSoup
import re

In [2]:
"""
# merge two pandas to one
data1 = pd.read_csv('data-1to500.csv')
data2 = pd.read_csv('data-500to1000.csv')
data_full = pd.concat([data1, data2])

# Because title1 and title2 is almost same, so drop one col
data_full['title'] = data_full['title1']
data_full.drop(columns=['title2', 'title1'], inplace=True)
# Save data_full to csv
data_full.to_csv('data_full.csv', index=None)
data_full.head()
"""
"""hihi"""

'hihi'

In [4]:
data = pd.read_csv('1to500.csv', encoding='utf-16')
print('Data Shape: ', data.shape)
data.head(5)

Data Shape:  (9972, 6)


,title1,title2,price,type,time,link
0,Bán nhà trọ 9 phòng 160m²,Bán nhà trọ 9 phòng 160m²,6.200.000.000 đ,Môi giới,2019-11-16 00:11:01.081813,https://nha.chotot.com//tp-ho-chi-minh/quan-go...
1,Bán nhà hẻm 4m p 16,Bán nhà hẻm 4m p 16,3.600.000.000 đ,Môi giới,2019-11-16 00:11:01.084112,https://nha.chotot.com//tp-ho-chi-minh/quan-go...
2,Bán nhà quang trung,Bán nhà quang trung,3.500.000.000 đ,Môi giới,2019-11-16 00:11:01.086491,https://nha.chotot.com//tp-ho-chi-minh/quan-go...
3,Bán gấp nhà hxh - 7mx37m - Xa Lộ Hà Nội - Thủ Đức,Bán gấp nhà hxh - 7mx37m - Xa Lộ Hà Nội - Thủ Đức,3.550.000.000 đ,TOMMY HUỲNH,2019-11-16 00:09:01.088832,https://nha.chotot.com//tp-ho-chi-minh/quan-th...
4,"Nhà hẻm 6m 489 Huỳnh Văn Bánh, 4x18, 4 lầu đẹp","Nhà hẻm 6m 489 Huỳnh Văn Bánh, 4x18, 4 lầu đẹp",10.990.000.000 đ,Môi giới,2019-11-16 00:08:01.091167,https://nha.chotot.com//tp-ho-chi-minh/quan-ph...


### After test for javascript, we saw that site use javascript to render page, so we decide to use selenium.webdriver to crawl information from attribute link.

In [5]:
"""
links = data.link[1000]
print(links)
ff = webdriver.Firefox()
ff.get(links)
r = HTML(html = ff.page_source)
ff.close()
"""
"""Code Test"""

'Code Test'

In [6]:
"""
print('Tiêu đề ', r.find('h1',first=True).text)
p_list = r.find('p')
descrip = [i for i in p_list if 'itemprop' in i.attrs and i.attrs['itemprop'] == 'description'][0]
descrip = BeautifulSoup(descrip.html, 'html.parser')
print('Mô tả chi tiết:', descrip.get_text())
tmp = r.find('.media-body.media-middle')[0]
print('Địa chỉ cụ thể:', tmp.text)
print()

l = r.find('._1DTCXk4eb6dusdylsQxCos', first=True)
for i in l.find('span'):
    print(i.text.split(': '))
    """
"""Code test"""

'Code test'

1. Số phòng ngủ
2. Diện tích
3. Loại hình nhà ở
4. Tiêu đề
5. Mô tả chi tiết
6. Số phòng vệ sinh
7. Hướng cửa chính
8. Giấy tờ pháp lý
9. Tên phân khu/lô/block/tháp
10. Số hiệu
11. Tổng số tầng
12. Đặc điểm nhà đất
13. Địa chỉ

Có những cái như `Tiêu đề, Mô tả chi tiết, Địa chỉ` ta có thể lấy riêng được, nên tạo một dict chứa những thuộc tính còn lại

In [5]:
def init_dict():
    return {'diện tích': None, 'số phòng ngủ': None, 'loại hình nhà ở': None, 'số phòng vệ sinh': None, 
            'hướng cửa chính': None, 'giấy tờ pháp lý': None, 'tên phân khu/lô/block/tháp': None, 'số hiệu':None, 
            'tổng số tầng':None, 'đặc điểm nhà/đất': None}

In [6]:
def get_description(r):
    p_list = r.find('p')
    descrip = [i for i in p_list if 'itemprop' in i.attrs and i.attrs['itemprop'] == 'description'][0]
    descrip = BeautifulSoup(descrip.html, 'html.parser')
    strgs = []
    for string in descrip.strings:
        strg = re.sub('\\xa0','', string)
        strg = re.sub('^ +','', strg)
        strg = re.sub(' +$', '', strg)
        strgs.append(strg)
    return '\n'.join(strgs)
# Test
# get_description(r)

In [7]:
tieu_de_list = []
mtct_list = []
diachi_list = []
dict_list = []

In [ ]:
cnt=1
for links in data.link:
    cnt+=1
    try:
        dicts = init_dict()
        # links = data.link[cnt]
        ff = webdriver.Chrome()
        ff.get(links)
        r = HTML(html = ff.page_source)
        ff.close()

        tit = r.find('h1', first=True).text
        desp = get_description(r)
        tmp = r.find('.media-body.media-middle')[0]
        addr = tmp.text


        l = r.find('._1DTCXk4eb6dusdylsQxCos', first=True)
        for i in l.find('span'):
            tmpd = i.text.split(': ')
            dicts[tmpd[0].lower()] = tmpd[1]

        if len(dicts) != len(init_dict()):
            print('Link: ', links, 'có thuộc tính mới, dict: ', dicts)

        tieu_de_list.append(tit)
        mtct_list.append(desp)
        diachi_list.append(addr)
        dict_list.append(dicts)
    except:
        print('Loi, link:', links)
        tieu_de_list.append(None)
        mtct_list.append(None)
        diachi_list.append(None)
        dict_list.append(init_dict())

Loi, link: https://nha.chotot.com//tp-ho-chi-minh/quan-thu-duc/mua-ban-nha-dat/64131123.htm
Loi, link: https://nha.chotot.com//tp-ho-chi-minh/quan-phu-nhuan/mua-ban-nha-dat/64262602.htm
Loi, link: https://nha.chotot.com//tp-ho-chi-minh/quan-10/mua-ban-nha-dat/64792962.htm
Loi, link: https://nha.chotot.com//tp-ho-chi-minh/quan-binh-tan/mua-ban-nha-dat/61894357.htm
Loi, link: https://nha.chotot.com//tp-ho-chi-minh/huyen-binh-chanh/mua-ban-nha-dat/62802830.htm
Loi, link: https://nha.chotot.com//tp-ho-chi-minh/quan-go-vap/mua-ban-nha-dat/64814361.htm
Loi, link: https://nha.chotot.com//tp-ho-chi-minh/quan-7/mua-ban-nha-dat/64355934.htm
Loi, link: https://nha.chotot.com//tp-ho-chi-minh/quan-6/mua-ban-nha-dat/64518779.htm
Loi, link: https://nha.chotot.com//tp-ho-chi-minh/quan-binh-tan/mua-ban-nha-dat/65143617.htm
Loi, link: https://nha.chotot.com//tp-ho-chi-minh/quan-phu-nhuan/mua-ban-nha-dat/64231863.htm
Loi, link: https://nha.chotot.com//tp-ho-chi-minh/huyen-nha-be/mua-ban-nha-dat/65296620.

Link:  https://nha.chotot.com//tp-ho-chi-minh/quan-thu-duc/mua-ban-nha-dat/65251599.htm có thuộc tính mới, dict:  {'diện tích': None, 'số phòng ngủ': None, 'loại hình nhà ở': None, 'số phòng vệ sinh': None, 'hướng cửa chính': None, 'giấy tờ pháp lý': None, 'tên phân khu/lô/block/tháp': None, 'số hiệu': None, 'tổng số tầng': None, 'đặc điểm nhà/đất': None, 'diá»\x87n tã\xadch': '113 mÂ²', 'sá»\x91 phã²ng ngá»§': '2 phÃ²ng', 'sá»\x91 phã²ng vá»\x87 sinh': '2 phÃ²ng', 'loáº¡i hã¬nh nhã\xa0 á»\x9f': 'NhÃ', 'giáº¥y tá»\x9d phã¡p lã½': 'Ä\x90Ã£ cÃ³ sá»\x95 Ä\x91á»\x8f/sá»\x95 há»\x93ng', 'ä\x90áº·c ä\x91iá»\x83m nhã\xa0/ä\x91áº¥t': 'Ná»\x9f háº\xadu'}
Loi, link: https://nha.chotot.com//tp-ho-chi-minh/quan-12/mua-ban-nha-dat/61498564.htm
Loi, link: https://nha.chotot.com//tp-ho-chi-minh/quan-go-vap/mua-ban-nha-dat/63641986.htm
Loi, link: https://nha.chotot.com//tp-ho-chi-minh/quan-6/mua-ban-nha-dat/55611245.htm
Loi, link: https://nha.chotot.com//tp-ho-chi-minh/quan-2/mua-ban-nha-dat/65292893.

Loi, link: https://nha.chotot.com//tp-ho-chi-minh/quan-3/mua-ban-nha-dat/65290774.htm
Loi, link: https://nha.chotot.com//tp-ho-chi-minh/quan-2/mua-ban-nha-dat/64963327.htm


In [ ]:
f = open('result1-500.csv', 'w', encoding='utf-16')
length = len(tieu_de_list)

for i in length:
    f.write('tieu_de_list[i]\ttct_list[i]\tdiachi_list\tdict_list[i]\n') # Write header to csv file
    
f.close()